# 📱 GERADOR DE APK SPY MOBILE

## 🚀 Execute as células na ordem e aguarde 20 minutos

### ⚡ Apenas 3 células para gerar seu APK!

In [ ]:
# 🔧 CÉLULA 1 - SETUP RÁPIDO (2 minutos)

# Instalar dependências
!apt update -qq
!apt install -y unzip wget
!pip install buildozer kivy requests

# Configurar Java (já instalado no Colab)
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-11-openjdk-amd64'

# Baixar Android SDK
!wget -q https://dl.google.com/android/repository/commandlinetools-linux-8512546_latest.zip -O /tmp/sdk.zip
!mkdir -p /content/android && cd /content/android && unzip -q /tmp/sdk.zip
!mkdir -p /content/android/cmdline-tools/latest
!mv /content/android/cmdline-tools/* /content/android/cmdline-tools/latest/ 2>/dev/null || true

# Configurar ambiente
os.environ.update({
    'ANDROID_HOME': '/content/android',
    'ANDROID_SDK_ROOT': '/content/android',
    'PATH': os.environ['PATH'] + ':/content/android/cmdline-tools/latest/bin'
})

print("✅ Setup concluído!")

In [ ]:
# ⚙️ CÉLULA 2 - CONFIGURAR SDK (1 minuto)

# Aceitar licenças e instalar componentes
!yes | /content/android/cmdline-tools/latest/bin/sdkmanager --licenses
!/content/android/cmdline-tools/latest/bin/sdkmanager "platforms;android-30" "build-tools;30.0.3"

print("✅ SDK configurado!")

In [ ]:
# 🔥 CÉLULA 3 - CRIAR E GERAR APK (15-20 minutos)

# Criar projeto
!mkdir -p /content/app && cd /content/app

# Criar main.py
with open('/content/app/main.py', 'w') as f:
    f.write('''from kivy.app import App
from kivy.uix.boxlayout import BoxLayout
from kivy.uix.label import Label
from kivy.uix.button import Button
from kivy.clock import Clock
from kivy.core.window import Window
import time
import requests

class SpyApp(BoxLayout):
    def __init__(self, **kwargs):
        super().__init__(orientation='vertical', **kwargs)
        self.is_monitoring = False
        self.start_time = None
        self.timer_event = None
        
        layout = BoxLayout(orientation='vertical', padding=20, spacing=20)
        
        layout.add_widget(Label(
            text='SPY MOBILE', 
            font_size=30, 
            color=(1,0,0,1), 
            bold=True
        ))
        
        self.timer_label = Label(
            text='00:00:00', 
            font_size=60, 
            color=(0.5,0.5,0.5,1),
            bold=True
        )
        layout.add_widget(self.timer_label)
        
        self.status_label = Label(
            text='Sistema Parado', 
            font_size=18,
            color=(0.7,0,0,1)
        )
        layout.add_widget(self.status_label)
        
        buttons = BoxLayout(orientation='horizontal', size_hint_y=0.3, spacing=10)
        
        self.play_btn = Button(
            text='PLAY', 
            font_size=20, 
            background_color=(0,0.8,0,1),
            bold=True
        )
        self.play_btn.bind(on_press=self.start)
        buttons.add_widget(self.play_btn)
        
        self.stop_btn = Button(
            text='STOP', 
            font_size=20, 
            background_color=(0.8,0,0,1),
            bold=True,
            disabled=True
        )
        self.stop_btn.bind(on_press=self.stop)
        buttons.add_widget(self.stop_btn)
        
        layout.add_widget(buttons)
        layout.add_widget(Label(
            text='Sistema de Monitoramento', 
            font_size=14,
            color=(0.5,0.5,0.5,1)
        ))
        
        self.add_widget(layout)

    def start(self, btn):
        if not self.is_monitoring:
            self.is_monitoring = True
            self.start_time = time.time()
            self.status_label.text = 'Monitorando...'
            self.timer_label.color = (1,0,0,1)
            self.play_btn.disabled = True
            self.stop_btn.disabled = False
            self.timer_event = Clock.schedule_interval(self.update_timer, 1)
            self.collect_data()

    def stop(self, btn):
        if self.is_monitoring:
            self.is_monitoring = False
            if self.timer_event:
                self.timer_event.cancel()
            self.status_label.text = 'Sistema Parado'
            self.timer_label.color = (0.5,0.5,0.5,1)
            self.timer_label.text = '00:00:00'
            self.play_btn.disabled = False
            self.stop_btn.disabled = True

    def update_timer(self, dt):
        if self.is_monitoring and self.start_time:
            elapsed = int(time.time() - self.start_time)
            h, m, s = elapsed//3600, (elapsed%3600)//60, elapsed%60
            self.timer_label.text = f'{h:02d}:{m:02d}:{s:02d}'

    def collect_data(self):
        try:
            data = {
                'device': 'spy_mobile',
                'timestamp': time.time(),
                'status': 'active'
            }
            
            # ALTERE O IP AQUI ⬇️
            try:
                response = requests.post(
                    'http://192.168.0.97:8000/api/data/', 
                    json=data, 
                    timeout=3
                )
                if response.status_code == 200:
                    self.status_label.text = 'Dados Enviados!'
                else:
                    self.status_label.text = 'Coletando (Offline)'
            except:
                self.status_label.text = 'Coletando (Offline)'
                
        except Exception as e:
            self.status_label.text = f'Erro: {str(e)[:30]}'

class SpyMobileApp(App):
    def build(self):
        Window.clearcolor = (0, 0, 0, 1)
        return SpyApp()

if __name__ == '__main__':
    SpyMobileApp().run()
''')

# Criar buildozer.spec
with open('/content/app/buildozer.spec', 'w') as f:
    f.write('''[app]
title = Spy Mobile
package.name = spymobile
package.domain = org.spy
source.dir = .
version = 1.0
requirements = python3,kivy,requests
orientation = portrait
android.api = 30
android.minapi = 21
android.archs = arm64-v8a
android.release_artifact = apk

[buildozer]
log_level = 1
warn_on_root = 0

[app:android.permissions]
INTERNET = 1
ACCESS_NETWORK_STATE = 1
ACCESS_FINE_LOCATION = 1
READ_EXTERNAL_STORAGE = 1
CAMERA = 1
''')

# Configurar ambiente final
import os
os.environ.update({
    'ANDROID_HOME': '/content/android',
    'ANDROID_SDK_ROOT': '/content/android',
    'JAVA_HOME': '/usr/lib/jvm/java-11-openjdk-amd64'
})

# Gerar APK
%cd /content/app
!buildozer init
print("🔥 GERANDO APK... Aguarde 15-20 minutos ⏰")
!buildozer android debug

# Verificar e copiar APK
import glob
apk_files = glob.glob('bin/*.apk')
if apk_files:
    !cp {apk_files[0]} /content/SpyMobile.apk
    import os
    size = os.path.getsize('/content/SpyMobile.apk') / (1024*1024)
    print(f"\n🎉 APK PRONTO! Tamanho: {size:.1f}MB")
    print("📱 Baixe: Pasta (📁) → SpyMobile.apk → (...) → Download")
else:
    print("❌ Erro na geração. Execute novamente.")

## 🎉 APK GERADO COM SUCESSO!

### 📱 COMO BAIXAR:
1. Clique no ícone de pasta (📁) no painel esquerdo
2. Procure pelo arquivo **SpyMobile.apk**
3. Clique nos 3 pontos (...) ao lado do arquivo
4. Selecione **Download**

### 📋 INSTALAÇÃO NO ANDROID:
1. Transfira o APK para seu dispositivo Android
2. Habilite **"Fontes desconhecidas"** nas configurações
3. Instale o APK normalmente
4. Abra o app e teste os botões PLAY/STOP

### ⚙️ CONFIGURAÇÕES:
- **Alterar IP do servidor**: Modifique a linha `http://192.168.0.97:8000/api/data/` na célula 3
- **Mudar nome do app**: Altere `title = Spy Mobile` no buildozer.spec
- **Adicionar permissões**: Edite a seção `[app:android.permissions]`

### 🔧 SE DER ERRO:
- Execute novamente a célula 3
- Reinicie o runtime se necessário
- Verifique se todas as células anteriores foram executadas

**✅ Seu APK está pronto para uso!**